In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

import numpy as np
import pytorch_lightning as pl
import torch
import datamol as dm

import goli

Using backend: pytorch


In [4]:
# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"

# Load your dataframe
df = goli.data.load_tiny_zinc()
df.head()

,SMILES,SA,logp,score
0,CCc1ccc(C(=O)/C(=C(/S)NC2CC2)[n+]2ccc(CC)cc2)cc1,2.775189,3.7897,6.564889
1,C=CCOc1ccc(C(F)(F)F)cc1C(=O)NC[C@H]1CCC[C@H]1O,3.071498,3.1610,6.232498
2,COc1cc(OC)cc([C@@H](NC(=O)Cn2ccccc2=O)c2nccn2C)c1,2.840009,1.5048,4.344809
3,CN1CCC[C@@]2(CCN(C(=O)CN3CCNC(=O)C3)C2)C1=O,3.605168,-1.1109,2.494268
4,COc1ccc(Cl)cc1NC(=O)c1nn(C)cc1[N+](=O)[O-],2.132665,2.2426,4.375265


In [5]:
# Not all the config keys must go in a config file. Some config
# have good default that will not change very often.

# Setup the featurization
featurization_args = {}
featurization_args["atom_property_list_float"] = []  # ["weight", "valence"]
featurization_args["atom_property_list_onehot"] = ["atomic-number", "degree"]
featurization_args["edge_property_list"] = ["ring", "bond-type-onehot"]
featurization_args["add_self_loop"] = False
featurization_args["use_bonds_weights"] = False
featurization_args["explicit_H"] = False

# Config for datamodule
dm_args = {}
dm_args["df"] = df
dm_args["cache_data_path"] = cache_data_path  # unsed at the moment
dm_args["featurization"] = featurization_args
dm_args["smiles_col"] = "SMILES"
dm_args["label_cols"] = None#["SA", "logp", "score"]
dm_args["split_val"] = 0.2
dm_args["split_test"] = 0.2
dm_args["split_seed"] = 19
dm_args["train_val_batch_size"] = 16
dm_args["test_batch_size"] = 16
dm_args["num_workers"] = 0
dm_args["pin_memory"] = True
dm_args["featurization_n_jobs"] = 16
dm_args["featurization_progress"] = True


dm = goli.data.DGLFromSmilesDataModule(**dm_args)
dm

In [11]:
# Load and prepare the data
dm.prepare_data()

# Create the split torch datasets
dm.setup()

2021-03-16 17:40:33.155 | INFO     | goli.data.datamodule:prepare_data:150 - Reload data from /tmp/tmp5wyleira/cache.pkl.


In [12]:
# Load a dataloader and get the first batch from it
dl = dm.train_dataloader()
it = iter(dl)
batch = next(it)
batch

{'smiles': ['c1cc2c(cc1N[C@@H]1CCOC3(CCC3)C1)CCC2',
  'COc1ccc(Cl)cc1NC(=O)c1nn(C)cc1[N+](=O)[O-]',
  'CC(=O)N1CC[C@@H]([NH2+][C@@H](C)CSCC(C)C)C1',
  'CCc1onc(C)c1NC(=O)CCCC(C)(C)C',
  'COC(C[C@@](C)(O)C[C@@H]1CCCN(S(C)(=O)=O)C1)OC',
  'CCN1C(=O)C(C#N)=C(C)/C(=C\\Nc2ccc([N+](=O)[O-])cc2)C1=O',
  'CCOC[C@H](O)[C@](C)(CC)[NH+]1CCCC1',
  'O=C(Nc1c[nH]c2ccccc12)N1CCCC[C@@H]1CCO',
  'O=C(CNc1nc(C2CC2)no1)N1CCc2sccc2C1',
  'CC(C)n1cccc1C(=O)N[C@H]1CCc2cc(N)ccc21',
  'Cc1ccsc1C(=O)NCCNC(=O)c1ccco1',
  'COc1ccc2c(c1)[C@H]([NH2+][C@H](C)CCN1CCOCC1)CCCO2',
  'CNC(=O)[C@@H]1CCC[NH+]1Cc1ccc(C)c(F)c1',
  'CC[NH+](C/C=C/c1ccc(C#N)cc1)C[C@H](C)C#N',
  'COC(=O)[C@H](C)N(Cc1ccccc1)C(=O)Cc1ccon1',
  'CC1(C)CCC[C@H]1n1c(N)[nH+]c2ccccc21'],
 'features': Graph(num_nodes=316, num_edges=670,
       ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}),
 'labels': tensor([[3.3631, 3.6889, 7.0520],
         [2.1327, 2.2426, 4.3